In [3]:
# Import Dependencies
import requests
import json
import pandas as pd
from config import yelp_key

In [4]:
# API Base URL
base_url = "https://api.yelp.com/v3/businesses/search?"

In [5]:
# API Headers
headers = {
    "accept" : "application/json",
    "Authorization" : "Bearer " + yelp_key 
}

In [20]:
# Create List of Categories (manually located)
categories = ['beaches','hanggliding','horsebackriding',
             'hiking','hot_air_baloons',
              'paddleboarding','parasailing',
              'sailing','snorkeling','ziplining', 
              'spas','hotsprings','massage','bedbreakfast','hotels','resorts',
              'skiresorts','tours', 'transportation','restaurants','nightlife'
             ]

len(categories)

21

In [ ]:
# Import Lat/Long Database
csv_path = "INSERT TAZIA'S CSV PATH HERE"
locations_df = pd.read_csv(csv_path)
locations_df.head()

In [ ]:
# Gather Lists of Latitudes and Longitudes
latitudes_list = []
longitudes_list = []
for i in range(locations_df.shape[0]):
    latitudes_list.append(locations_df.at[i,**INSERT COLUMN NUM FOR LATITUDE**])
    longitudes_list.append(locations_df.at[i,**INSERT COLUMN NUM FOR LONGITUDE**])
print(len(latitudes_list))
print(len(longitudes_list))

In [ ]:
# Gather List of URLs
url_list = []
lat_lon_list = []
cat_list = []
for i in len(latitudes_list):
        for x in len(categories):
            latitude = latitudes_list[i]
            longitude = longitudes_list[i]
            category = categories[x]
            # Limit radius to appx 15 miles from the coordinates
            complete_url = base_url + "latitude=" + latitude + "&longitude=" + longitude + "&radius=24140&categories=" + category + "&sort_by=best_match"
            url_list.append(complete_url)
            lat_lon_list.append([latitude,longitude])
            cat_list.append(category)
# Confirm list of API calls is less than 5,000 daily limit
print(len(url_list))
url_list

In [ ]:
# Create API Calls
data_list=[]
for i in len(url_list):
    response = requests.get(url_list[i],headers=headers)
    data = response.json()
    
    # Calculate Average Rating and Review Count
    rating_sum = 0
    review_sum = 0
    for x in range(len(data["businesses"])):
        rating_sum += data["businesses"][x]["rating"]
        review_sum += data["businesses"][x]["review_count"]
    rating_avg = rating_sum / len(data)
    review_avg = review_sum / len(data)
    
    # Add the desired data to a dictionary
    data_dict = {
        "latitude" : lat_lon_list[i][0] ,
        "longitude": lat_lon_list[i][1],
        "category" : cat_list[i],
        "results_total" : len(data["businesses"]),
        "avg_rating" : rating_avg,
        "avg_review_count" : review_avg
    }
    # Append the dictionary to a list
    data_list.append(data_dict)
    
yelp_df = pd.DataFrame(data_list)   
yelp_df